# Proyecto 

In [2]:
import pandas as pd
import numpy as np

import random
from scipy.stats import truncnorm
import uuid

### Requesito 1

In [99]:
def generate_unique_id(n):
    return [str(uuid.uuid4()).replace('-', '')[:16] for _ in range(n)]

def generate_unique_id_1(n):
    return [str(uuid.uuid4()).replace('-', '')[:16] for _ in range((len((d1['columns'][0]['values'])))
)]

In [111]:
d1 = {
    "ds": "dataset1",
    "columns": [
        {
            "name": "area",
            "type": "category",
            "values": ["TI", "FIN", "HR"]
        },
        {
            "name": "id",
            "type": "unique",
            "values": generate_unique_id_1(3)
        }
    ],
    "random": False
}

In [109]:
# Crear el dataframe del segundo dataset
data1 = {
    "area": d2["columns"][2]["values"],
    "id": generate_unique_id(3),
}

data = {
    d1["columns"][0]["name"]: d1["columns"][0]["values"],
    d1["columns"][1]["name"]: d1["columns"][1]["values"]
}

df = pd.DataFrame(data)
df

,area,id
0,TI,f1766810b1c64d42
1,FIN,dd5c383de262410a
2,HR,9773929f296e4d3b


## Requisito 2

In [15]:
d2 = {
    "ds": "dataset2",
    "columns": [
        {
            "name": "id",
            "type": "unique",
            "values": "generate_unique_id(4)"
        },
        {
            "name": "area",
            "type": "foreign",
            "values": "dataset1.area"
        },
        {
            "name": "subarea",
            "type": "category",
            "values": ["SA1", "SA2", "SA3", "SA4"]
        }
    ],
    "random": False
}


In [17]:
# Crear dataset1 basado en el diccionario d1
d1["id"] = d1["columns"][1]["values"]


In [19]:
# Verificacion
referenced_dataset = d1  # Referencia al dataset 1

# Verificar que el dataset de referencia exista
if d2["columns"][1]["values"].split('.')[0] != referenced_dataset['ds']:
    raise ValueError(f"Referencia inválida al dataset. Dataset válido: {referenced_dataset['ds']}")

# Verificar que la columna de referencia exista
if d2["columns"][1]["values"].split('.')[1] not in [col['name'] for col in referenced_dataset['columns']]:
    raise ValueError(f"Columna de referencia inválida. Columnas válidas: {[col['name'] for col in referenced_dataset['columns']]}")

# Generar IDs únicos para d2
def generate_unique_id_d2():
    return [str(uuid.uuid4()).replace('-', '')[:16] for _ in range(4)]  # 4 registros en d2

unique_ids_d2 = generate_unique_id_d2()

# Selección aleatoria de valores para la columna 'area' del dataset 1
random_areas = [random.choice(referenced_dataset["columns"][0]["values"]) for _ in range(4)]

# Crear el dataframe del segundo dataset
data2 = {
    "id": unique_ids_d2,
    "area": random_areas,
    "subarea": d2["columns"][2]["values"]
}

df2 = pd.DataFrame(data2)

# Imprimir el segundo dataset
print("Dataset 2:")
df2 = pd.DataFrame(data2)
df2


Dataset 2:


,id,area,subarea
0,05c036d1e2444609,TI,SA1
1,ef91760e020d408c,FIN,SA2
2,52a9f50dc3d24a36,TI,SA3
3,363a118539d84893,HR,SA4


## Requisito 3

In [12]:
d3 = {
    "ds": "dataset3",
    "columns": [
        {
            "name": "id",
            "type": "unique",
            "values": generate_unique_id(1000)
        },
        {
            "name": "subarea",
            "type": "category",
            "values": "dataset2.id"
        },
        {
            "name": "income",
            "type": "numeric",
            "values": {
                "min": 1000,
                "max": 10000
            }
        },
        {
            "name": "goal",
            "type": "numeric",
            "values": {
                "min": 1000,
                "max": 10000,
                "std": 5000,
                "mean": 5000
            }
        }
    ],
    "random": True,
    "random_rows": 1000
}


In [13]:
# d3 ya tiene los valores definidos
mean = d3["columns"][3]["values"]["mean"]
std = d3["columns"][3]["values"]["std"]
min_val = d3["columns"][3]["values"]["min"]
max_val = d3["columns"][3]["values"]["max"]
size = d3["random_rows"]  # Tamaño de muestra

# Definir la función para generar datos
def generate_truncated_normal_data(mean, std, min_val, max_val, size):
    # Calcular los parámetros de la distribución normal truncada
    a, b = (min_val - mean) / std, (max_val - mean) / std
    data = truncnorm(a, b, loc=mean, scale=std).rvs(size)
    return data

# Generar los datos
data = generate_truncated_normal_data(mean, std, min_val, max_val, size)

referenced_dataset = d2  # Referencia al dataset 1
random_rows = 1000
# Selección aleatoria de valores para la columna 'area' del dataset 1
random_areas = [random.choice(referenced_dataset["columns"][1]["values"]) for _ in range(random_rows)]

def generate_normal_data_in_range(min_val, max_val, size):
    mean = (min_val + max_val) / 2
    std = (max_val - min_val) / 6  # Aproximadamente 99.7% de los datos caerán dentro de [min_val, max_val]
    data = np.random.normal(loc=mean, scale=std, size=size)
    return np.clip(data, min_val, max_val)


In [59]:
# Generar dataset3 con 'subarea' referenciado desde data2.id
n_rows = 1000
dataset3 = pd.DataFrame({
    "id": generate_unique_id(1000),  # Generar 1000 IDs únicos
    "subarea": np.random.choice(data2["id"], n_rows),  # Elegir aleatoriamente de los IDs en data2
    "income": np.random.uniform(1000, 5000, n_rows),  # Generar valores numéricos aleatorios para 'income'
    "goal": np.random.normal(5000, 1000, n_rows)  # Generar valores numéricos aleatorios para 'goal'
})

In [61]:
df3 = pd.DataFrame(dataset3)
df3

,id,subarea,income,goal
0,ae804b38949d452b,363a118539d84893,2562.657337,6366.426103
1,38702e914d7b4dbc,52a9f50dc3d24a36,2202.495893,4489.798623
2,6e809ebabbf04f83,363a118539d84893,3935.270003,4580.739456
3,c4df54f283054660,363a118539d84893,2730.879217,5401.001733
4,7ba216d35974450e,363a118539d84893,4133.729318,6335.093570
...,...,...,...,...
995,37802527e9b84581,05c036d1e2444609,3916.932100,3672.122200
996,f35f5707c04d4d01,52a9f50dc3d24a36,1120.752845,2389.832811
997,f137a086a7184db7,363a118539d84893,1953.198058,5142.561033
998,2a51e13e19dc4c70,ef91760e020d408c,1530.196529,4781.418660


## Requisito 4

In [ ]:
conf_list = [d5, d2, d4, d1, d3]
def build_dataframes(conf_list):
    dataframes = {}
    foreign_columns = []

    # Primera pasada: crear dataframes sin columnas 'foreign'
    for conf in conf_list:
        if "ds" not in conf:
            raise KeyError(f"La configuración {conf} no tiene la clave 'ds'.")
        if "columns" not in conf:
            raise KeyError(f"La configuración {conf} no tiene la clave 'columns'.")

        ds_name = conf["ds"]
        columns = conf["columns"]
        df_data = {}

        # Encontrar la longitud correcta para cada dataset
        length = conf.get("random_rows", 5)
        if not conf["random"]:
            length = max(len(col["values"]) for col in columns if col["type"] in ["unique", "category", "text"])

        for col in columns:
            col_name = col["name"]
            col_type = col["type"]
            if col_type == "unique":
                df_data[col_name] = generate_unique_id(length)
            elif col_type == "category":
                if isinstance(col["values"], str) and "." in col["values"]:
                    foreign_ds, foreign_col = col["values"].split(".")
                    foreign_columns.append((ds_name, col_name, foreign_ds, foreign_col, length))
                elif isinstance(col["values"], list) and all(isinstance(i, str) for i in col["values"]):
                    df_data[col_name] = np.random.choice(col["values"], length)
                else:
                    raise ValueError(f"Los valores de la columna '{col_name}' en el dataset '{ds_name}' no son válidos.")
            elif col_type == "numeric":
                if "min" in col["values"] and "max" in col["values"] and "std" in col["values"] and "mean" in col["values"]:
                    # Generar datos utilizando una distribución normal truncada
                    df_data[col_name] = generate_truncated_normal_data(
                        col["values"]["mean"],
                        col["values"]["std"],
                        col["values"]["min"],
                        col["values"]["max"],
                        length
                    )
                elif "min" in col["values"] and "max" in col["values"]:
                    # Generar datos utilizando una distribución normal simple dentro de un rango
                    df_data[col_name] = generate_normal_data_in_range(
                        col["values"]["min"],
                        col["values"]["max"],
                        length
                    )
                else:
                    raise ValueError(f"Los valores de la columna '{col_name}' en el dataset '{ds_name}' no son válidos.")

        dataframes[ds_name] = pd.DataFrame(df_data)

    # Segunda pasada: añadir columnas 'foreign'
    for ds_name, col_name, foreign_ds, foreign_col, length in foreign_columns:
        if foreign_ds not in dataframes:
            raise KeyError(f"El dataset '{foreign_ds}' no ha sido creado.")
        dataframes[ds_name][col_name] = np.random.choice(dataframes[foreign_ds][foreign_col].values, length)

    return dataframes

# Construcción de dataframes
dataframes = build_dataframes(conf_list)

# Convertir el diccionario a una lista de dataframes
dataframe_list = list(dataframes.values())

# Imprimir el dataframe correspondiente a dataset2
print(dataframe_list[4])

In [ ]:
d4 = {
    "ds": "dataset4",
    "columns": [
        {
            "name": "id",
            "type": "unique",
            "values": generate_unique_id(5)
        },
        {
            "name": "task",
            "type": "category",
            "values": ["Task1", "Task2", "Task3"]
        },
        {
            "name": "subarea",
            "type": "foreign",
            "values": "dataset2.subarea"
        }
    ],
    "random": False
}

d5 = {
    "ds": "dataset5",
    "columns": [
        {
            "name": "id",
            "type": "unique",
            "values": generate_unique_id(5)
        },
        {
            "name": "description",
            "type": "text",
            "values": ["Desc1", "Desc2", "Desc3"]
        },
        {
            "name": "task",
            "type": "foreign",
            "values": "dataset4.task"
        }
    ],
    "random": False
}

conf_list = [d5, d2, d4, d1, d3]

In [ ]:
all_dataframes = list(dataframes.values())
all_dataframes[2]

## Requisito 5

In [ ]:
print(df.columns)

In [ ]:
df5 = all_dataframes[0]  # Este es el dataset generado

# Analiza combinaciones de columnas categóricas
category_cols = ['id', 'area']  # Reemplaza con los nombres reales de las columnas categóricas
combinations = df[category_cols].value_counts(normalize=True).reset_index()
combinations.columns = category_cols + ['probability']

# Imprimir combinaciones y probabilidades
print(combinations)

In [ ]:
def simulate_categorical_data(combinations, num_records):
    simulated_data = []
    for _ in range(num_records):
        # Elige una combinación de acuerdo a sus probabilidades
        combination = combinations.sample(weights='probability').iloc[0]
        simulated_data.append(combination[:-1].values)  # Excluye la probabilidad
    return pd.DataFrame(simulated_data, columns=combinations.columns[:-1])

final_simulation = simulate_categorical_data(combinations, 100000)

In [ ]:
def simulate_numeric_column(mean, std, size):
    return np.random.normal(loc=mean, scale=std, size=size)

numeric_cols = ['mean', 'std']  # Reemplaza con columnas numéricas
for col in numeric_cols:
    mean = 50000  
    std = 10000  
    final_simulation[col] = simulate_numeric_column(mean, std, 100000)


In [ ]:
print( final_simulation )

In [ ]:
print("Categorical Simulation Result:")
print(final_simulation.head())